<a href="https://colab.research.google.com/github/GrueneKatze/AI-for-Medicine-Coursera/blob/master/Churn_rate_preds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Churn rate prediction with machine and deep learning models

https://www.kaggle.com/jinudaniel/telecom-churn-fast-ai-rf (fastai)

https://www.kaggle.com/pavanraj159/telecom-customer-churn-prediction (EDA!)

https://www.kaggle.com/danwheble/churn-prediction-telco-customer-churn

In [1]:
!curl -s https://course.fast.ai/setup/colab | bash

Updating fastai...
Done.


In [2]:
%matplotlib inline
from fastai.basics import *

In [3]:
from keras.utils import to_categorical


Using TensorFlow backend.


In [4]:
import numpy as np 
import pandas as pd

#visualization
import seaborn as sns
import matplotlib.pyplot as plt

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [5]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"
base_dir = root_dir + 'fastai/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
path_r = Path("/content/data") #Path(base_dir + 'rsna')
#path_r.mkdir(parents=True, exist_ok=True)

In [6]:
path_cr = Path(base_dir + 'churn_rate_t')
path_cr.mkdir(parents=True, exist_ok=True)

In [7]:
path_cr.ls()

[PosixPath('/content/gdrive/My Drive/fastai/churn_rate_t/datasets_13996_18858_WA_Fn-UseC_-Telco-Customer-Churn.csv')]

In [8]:
dataset = pd.read_csv(path_cr/'datasets_13996_18858_WA_Fn-UseC_-Telco-Customer-Churn.csv')

In [ ]:
dataset.sample()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
6829,8148-WOCMK,Male,0,Yes,Yes,8,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Credit card (automatic),19.6,125,No


In [9]:
dataset.shape

(7043, 21)

In [ ]:
dataset.dtypes

customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

In [ ]:
for i in dataset.columns:
  print(i)
  print(dataset[i].unique())

In [10]:
#excluding empty cells
dataset_wo_empty = dataset[dataset['TotalCharges'] != ' ']

In [11]:
dataset_wo_empty.shape

(7032, 21)

In [12]:
dataset_wo_empty['TotalCharges'] = pd.to_numeric(dataset_wo_empty['TotalCharges'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [13]:
dataset_wo_empty.dtypes

customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges        float64
Churn                object
dtype: object

In [19]:
#converting categorical to binary cols
dummy_fields = ['gender', 'Partner', 'Dependents','PhoneService','MultipleLines','InternetService','OnlineSecurity','OnlineBackup','DeviceProtection','TechSupport','StreamingTV','StreamingMovies','Contract','PaperlessBilling','PaymentMethod']
for each in dummy_fields:
    dummies = pd.get_dummies(dataset_wo_empty[each], prefix=each, drop_first=False)
    dataset_wo_empty = pd.concat([dummies, dataset_wo_empty], axis=1)

data_w_dummies = dataset_wo_empty.drop(dummy_fields, axis=1)
data_w_dummies.sample()
#dataset_w_dummies = pd.get_dummies(dataset_wo_empty)

,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,PaperlessBilling_No,PaperlessBilling_Yes,Contract_Month-to-month,Contract_One year,Contract_Two year,StreamingMovies_No,StreamingMovies_No internet service,StreamingMovies_Yes,StreamingTV_No,StreamingTV_No internet service,StreamingTV_Yes,TechSupport_No,TechSupport_No internet service,TechSupport_Yes,DeviceProtection_No,DeviceProtection_No internet service,DeviceProtection_Yes,OnlineBackup_No,OnlineBackup_No internet service,OnlineBackup_Yes,OnlineSecurity_No,OnlineSecurity_No internet service,OnlineSecurity_Yes,InternetService_DSL,InternetService_Fiber optic,InternetService_No,MultipleLines_No,MultipleLines_No phone service,MultipleLines_Yes,PhoneService_No,PhoneService_Yes,Dependents_No,Dependents_Yes,Partner_No,Partner_Yes,gender_Female,gender_Male,customerID,SeniorCitizen,tenure,MonthlyCharges,TotalCharges,Churn
4026,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,1,0,0,0,1,0,1,0,1,0,1,0,1,0266-GMEAO,0,72,114.3,8058.55,No


In [20]:
#Replacing Yes/ No with ones/ zeros
data_w_dummies.loc[data_w_dummies['Churn'] == 'Yes', 'Churn'] = 1
data_w_dummies.loc[data_w_dummies['Churn'] == 'No', 'Churn'] = 0

In [27]:
len(data_w_dummies.columns)
data_w_dummies.sample(5)

,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,PaperlessBilling_No,PaperlessBilling_Yes,Contract_Month-to-month,Contract_One year,Contract_Two year,StreamingMovies_No,StreamingMovies_No internet service,StreamingMovies_Yes,StreamingTV_No,StreamingTV_No internet service,StreamingTV_Yes,TechSupport_No,TechSupport_No internet service,TechSupport_Yes,DeviceProtection_No,DeviceProtection_No internet service,DeviceProtection_Yes,OnlineBackup_No,OnlineBackup_No internet service,OnlineBackup_Yes,OnlineSecurity_No,OnlineSecurity_No internet service,OnlineSecurity_Yes,InternetService_DSL,InternetService_Fiber optic,InternetService_No,MultipleLines_No,MultipleLines_No phone service,MultipleLines_Yes,PhoneService_No,PhoneService_Yes,Dependents_No,Dependents_Yes,Partner_No,Partner_Yes,gender_Female,gender_Male,customerID,SeniorCitizen,tenure,MonthlyCharges,TotalCharges,Churn
930,1,0,0,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,1,1,0,0,0,1,0,1,0,0,0,1,1,0,1,0,1,0,5146-CBVOE,0,16,75.10,1212.85,0
1126,0,0,0,1,1,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,1,1,0,0,1,0,0,0,1,0,1,0,0,0,1,1,0,1,0,0,1,0927-LCSMG,0,7,74.65,544.55,1
6397,1,0,0,0,1,0,0,1,0,0,0,1,1,0,0,0,0,1,0,0,1,1,0,0,1,0,0,1,0,0,1,0,0,0,1,1,0,1,0,0,1,7994-XIRTR,1,54,65.25,3529.95,0
232,0,0,0,1,0,1,1,0,0,0,0,1,1,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,1,0,1,0,0,0,1,1,0,1,0,0,1,8937-RDTHP,0,1,84.60,84.60,1
1749,0,0,1,0,0,1,1,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,1,0,0,0,1,0,0,0,1,0,1,1,0,1,0,0,1,1320-REHCS,1,52,110.75,5832.00,0


In [25]:
#len(['instant','dteday', 'season', 'yr', 'mnth', 'hr', 'holiday', 'weekday', 'workingday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed'])

14

In [21]:
corrmat = data_w_dummies.corr()

In [30]:
cols = ['PaymentMethod_Bank transfer (automatic)',
       'PaymentMethod_Credit card (automatic)',
       'PaymentMethod_Electronic check', 'PaymentMethod_Mailed check',
       'PaperlessBilling_No', 'PaperlessBilling_Yes',
       'Contract_Month-to-month', 'Contract_One year', 'Contract_Two year',
       'StreamingMovies_No', 'StreamingMovies_No internet service',
       'StreamingMovies_Yes', 'StreamingTV_No',
       'StreamingTV_No internet service', 'StreamingTV_Yes', 'TechSupport_No',
       'TechSupport_No internet service', 'TechSupport_Yes',
       'DeviceProtection_No', 'DeviceProtection_No internet service',
       'DeviceProtection_Yes', 'OnlineBackup_No',
       'OnlineBackup_No internet service', 'OnlineBackup_Yes',
       'OnlineSecurity_No', 'OnlineSecurity_No internet service',
       'OnlineSecurity_Yes', 'InternetService_DSL',
       'InternetService_Fiber optic', 'InternetService_No', 'MultipleLines_No',
       'MultipleLines_No phone service', 'MultipleLines_Yes',
       'PhoneService_No', 'PhoneService_Yes', 'Dependents_No',
       'Dependents_Yes', 'Partner_No', 'Partner_Yes', 'gender_Female',
       'gender_Male', 'SeniorCitizen', 'tenure',
       'MonthlyCharges', 'TotalCharges']
k = 47 #number of variables for heatmap
cols = corrmat.nlargest(k, 'Churn')['Churn'].index
cm = np.corrcoef(data_w_dummies[cols].values.T)
sns.set(font_scale=1.25)
hm = sns.heatmap(cm, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=cols.values, xticklabels=cols.values)
plt.show()

KeyError: ignored

## To del

In [ ]:
cols = ['customerID','gender', 'Partner', 'Dependents','PhoneService','MultipleLines','InternetService','OnlineSecurity','OnlineBackup','DeviceProtection','TechSupport','StreamingTV','StreamingMovies','Contract','PaperlessBilling','PaymentMethod','Churn']
for i in cols:
  dataset[i]= dataset[i].str.lower()

In [ ]:
dataset.sample(5)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
6296,4023-rtiqm,female,1,yes,no,31,no,no phone service,dsl,yes,yes,yes,no,no,yes,one year,yes,credit card (automatic),50.40,1580.1,no
4481,8644-xytsv,male,0,yes,no,42,no,no phone service,dsl,yes,no,no,no,yes,no,month-to-month,yes,bank transfer (automatic),40.15,1626.05,no
6848,5916-qewpt,female,0,yes,no,2,yes,no,dsl,no,no,yes,no,yes,yes,month-to-month,no,credit card (automatic),69.00,147.8,no
3636,6319-qsusr,female,0,no,yes,11,yes,no,no,no internet service,no internet service,no internet service,no internet service,no internet service,no internet service,month-to-month,no,credit card (automatic),19.70,216.2,no
2715,0484-jpbru,male,0,no,no,41,yes,yes,no,no internet service,no internet service,no internet service,no internet service,no internet service,no internet service,month-to-month,yes,bank transfer (automatic),25.25,996.45,no


In [ ]:
#dataset[dataset['Partner']].to_categorical()#(a, 2)

In [ ]:
data_w_dummies.shape

(7043, 47)

In [ ]:
data_w_dummies.isnull().any()

PaymentMethod_Bank transfer (automatic)    False
PaymentMethod_Credit card (automatic)      False
PaymentMethod_Electronic check             False
PaymentMethod_Mailed check                 False
PaperlessBilling_No                        False
PaperlessBilling_Yes                       False
Contract_Month-to-month                    False
Contract_One year                          False
Contract_Two year                          False
StreamingMovies_No                         False
StreamingMovies_No internet service        False
StreamingMovies_Yes                        False
StreamingTV_No                             False
StreamingTV_No internet service            False
StreamingTV_Yes                            False
TechSupport_No                             False
TechSupport_No internet service            False
TechSupport_Yes                            False
DeviceProtection_No                        False
DeviceProtection_No internet service       False
DeviceProtection_Yes

In [ ]:
data_w_dummies.isnull().sum(axis = 0)

PaymentMethod_Bank transfer (automatic)    0
PaymentMethod_Credit card (automatic)      0
PaymentMethod_Electronic check             0
PaymentMethod_Mailed check                 0
PaperlessBilling_No                        0
PaperlessBilling_Yes                       0
Contract_Month-to-month                    0
Contract_One year                          0
Contract_Two year                          0
StreamingMovies_No                         0
StreamingMovies_No internet service        0
StreamingMovies_Yes                        0
StreamingTV_No                             0
StreamingTV_No internet service            0
StreamingTV_Yes                            0
TechSupport_No                             0
TechSupport_No internet service            0
TechSupport_Yes                            0
DeviceProtection_No                        0
DeviceProtection_No internet service       0
DeviceProtection_Yes                       0
OnlineBackup_No                            0
OnlineBack

In [ ]:
data_w_dummies['TotalCharges'][488]

' '

In [ ]:
#Checking for empty cells
#np.where(data_w_dummies.applymap(lambda x: x == ''))
data_w_dummies[data_w_dummies['TotalCharges'] == ' '].index

In [ ]:
data_w_dummies.loc[data_w_dummies['TotalCharges'] == ' ', 'TotalCharges'] = 0

In [ ]:
data_w_dummies.loc[data_w_dummies['Churn'] == 'Yes', 'Churn'] = 1

In [ ]:
data_w_dummies.loc[data_w_dummies['Churn'] == 'No', 'Churn'] = 0

In [ ]:
data_w_dummies.sample(3)

,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,PaperlessBilling_No,PaperlessBilling_Yes,Contract_Month-to-month,Contract_One year,Contract_Two year,StreamingMovies_No,StreamingMovies_No internet service,StreamingMovies_Yes,StreamingTV_No,StreamingTV_No internet service,StreamingTV_Yes,TechSupport_No,TechSupport_No internet service,TechSupport_Yes,DeviceProtection_No,DeviceProtection_No internet service,DeviceProtection_Yes,OnlineBackup_No,OnlineBackup_No internet service,OnlineBackup_Yes,OnlineSecurity_No,OnlineSecurity_No internet service,OnlineSecurity_Yes,InternetService_DSL,InternetService_Fiber optic,InternetService_No,MultipleLines_No,MultipleLines_No phone service,MultipleLines_Yes,PhoneService_No,PhoneService_Yes,Dependents_No,Dependents_Yes,Partner_No,Partner_Yes,gender_Female,gender_Male,customerID,SeniorCitizen,tenure,MonthlyCharges,TotalCharges,Churn
3456,0,0,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,1,0,0,0,0,1,1,0,0,0,0,1,0,1,1,0,1,0,1,0,0436-TWFFZ,0,67,85.25,5714.2,0
666,0,0,1,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,1,0,0,0,1,1,0,1,0,1,0,7538-GWHML,0,2,69.65,145.15,1
5957,0,0,1,0,1,0,1,0,0,0,0,1,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,1,0,0,0,1,1,0,0,1,0,1,6877-LGWXO,1,18,78.55,1422.65,1


In [ ]:
data_w_dummies.dtypes

PaymentMethod_Bank transfer (automatic)      uint8
PaymentMethod_Credit card (automatic)        uint8
PaymentMethod_Electronic check               uint8
PaymentMethod_Mailed check                   uint8
PaperlessBilling_No                          uint8
PaperlessBilling_Yes                         uint8
Contract_Month-to-month                      uint8
Contract_One year                            uint8
Contract_Two year                            uint8
StreamingMovies_No                           uint8
StreamingMovies_No internet service          uint8
StreamingMovies_Yes                          uint8
StreamingTV_No                               uint8
StreamingTV_No internet service              uint8
StreamingTV_Yes                              uint8
TechSupport_No                               uint8
TechSupport_No internet service              uint8
TechSupport_Yes                              uint8
DeviceProtection_No                          uint8
DeviceProtection_No internet se

In [ ]:
data_w_dummies[480:492]

,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,PaperlessBilling_No,PaperlessBilling_Yes,Contract_Month-to-month,Contract_One year,Contract_Two year,StreamingMovies_No,StreamingMovies_No internet service,StreamingMovies_Yes,StreamingTV_No,StreamingTV_No internet service,StreamingTV_Yes,TechSupport_No,TechSupport_No internet service,TechSupport_Yes,DeviceProtection_No,DeviceProtection_No internet service,DeviceProtection_Yes,OnlineBackup_No,OnlineBackup_No internet service,OnlineBackup_Yes,OnlineSecurity_No,OnlineSecurity_No internet service,OnlineSecurity_Yes,InternetService_DSL,InternetService_Fiber optic,InternetService_No,MultipleLines_No,MultipleLines_No phone service,MultipleLines_Yes,PhoneService_No,PhoneService_Yes,Dependents_No,Dependents_Yes,Partner_No,Partner_Yes,gender_Female,gender_Male,customerID,SeniorCitizen,tenure,MonthlyCharges,TotalCharges,Churn
480,0,0,0,1,1,0,0,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,1,0,0,0,1,0,1,0,1,0,1,0486-LGCCH,0,11,19.65,225.75,0
481,1,0,0,0,0,1,1,0,0,0,0,1,1,0,0,1,0,0,0,0,1,1,0,0,0,0,1,1,0,0,0,1,0,1,0,1,0,0,1,1,0,0447-BEMNG,0,48,45.30,2145,1
482,0,0,1,0,0,1,1,0,0,0,0,1,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,1,0,0,0,1,1,0,1,0,1,0,4612-SSVHJ,1,20,81.45,1671.6,0
483,1,0,0,0,0,1,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,1,0,0,0,1,0,0,0,1,0,1,1,0,0,1,1,0,5168-MQQCA,0,72,108.50,8003.8,0
484,0,0,1,0,0,1,1,0,0,1,0,0,0,0,1,1,0,0,1,0,0,0,0,1,1,0,0,0,1,0,1,0,0,0,1,0,1,0,1,1,0,5949-XIKAE,0,8,83.55,680.05,1
485,0,1,0,0,0,1,0,0,1,0,0,1,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,1,0,1,0,1,0,1,7971-HLVXI,0,72,84.50,6130.85,0
486,0,0,1,0,0,1,1,0,0,0,0,1,0,0,1,1,0,0,1,0,0,0,0,1,1,0,0,0,1,0,0,0,1,0,1,1,0,1,0,1,0,9094-AZPHK,0,15,100.15,1415,0
487,1,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,1,0,0,1,0,0,0,0,1,0,1,1,0,1,0,0,1,3649-JPUGY,0,72,88.60,6201.95,0
488,1,0,0,0,0,1,0,0,1,1,0,0,0,0,1,0,0,1,0,0,1,1,0,0,0,0,1,1,0,0,0,1,0,1,0,0,1,0,1,1,0,4472-LVYGI,0,0,52.55,0,0
489,0,0,1,0,0,1,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,1,0,1,1,0,0,1,8372-JUXUI,0,1,74.35,74.35,1


In [ ]:
#data_w_dummies.replace(r'\s+', np.nan, regex=True)


In [ ]:
data_w_dummies['TotalCharges'] = pd.to_numeric(data_w_dummies['TotalCharges'])
#data_w_dummies["Churn"] = data_w_dummies.Churn.astype(float)